# Sentiment Classification


### Generate Word Embeddings and retrieve outputs of each layer with Keras based on Classification task

Word embeddings are a type of word representation that allows words with similar meaning to have a similar representation.

It is a distributed representation for text that is perhaps one of the key breakthroughs for the impressive performance of deep learning methods on challenging natural language processing problems.

We willl use the imdb dataset to learn word embeddings as we train our dataset. This dataset contains 25,000 movie reviews from IMDB, labeled with sentiment (positive or negative). 



### Dataset

`from keras.datasets import imdb`

Dataset of 25,000 movies reviews from IMDB, labeled by sentiment (positive/negative). Reviews have been preprocessed, and each review is encoded as a sequence of word indexes (integers). For convenience, the words are indexed by their frequency in the dataset, meaning the for that has index 1 is the most frequent word. Use the first 20 words from each review to speed up training, using a max vocab size of 10,000.

As a convention, "0" does not stand for a specific word, but instead is used to encode any unknown word.


### Aim

1. Import test and train data  
2. Import the labels ( train and test) 
3. Get the word index and then Create key value pair for word and word_id. (12.5 points)
4. Build a Sequential Model using Keras for Sentiment Classification task. (10 points)
5. Report the Accuracy of the model. (5 points)  
6. Retrive the output of each layer in keras for a given single test sample from the trained model you built. (2.5 points)


#### Usage:

In [1]:
from keras.datasets import imdb

vocab_size = 10000 #vocab size
#load dataset as a list of ints
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size) # vocab_size is no.of words to consider from the dataset, ordering based on frequency.

Using TensorFlow backend.


In [2]:
from keras.preprocessing.sequence import pad_sequences
maxlen = 300  #number of word used from each review

In [3]:
#(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)
#make all sequences of the same length
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test =  pad_sequences(x_test, maxlen=maxlen)

In [12]:
import keras
words = keras.datasets.imdb.get_word_index()
#The below step is redundant as get_word_index itself returns key value pair only 
words = {key:value for key,value in words.items()}   

In [13]:
words['the']

1

## Build Keras Embedding Layer Model
We can think of the Embedding layer as a dicionary that maps a index assigned to a word to a word vector. This layer is very flexible and can be used in a few ways:

* The embedding layer can be used at the start of a larger deep learning model. 
* Also we could load pre-train word embeddings into the embedding layer when we create our model.
* Use the embedding layer to train our own word2vec models.

The keras embedding layer doesn't require us to onehot encode our words, instead we have to give each word a unqiue intger number as an id. For the imdb dataset we've loaded this has already been done, but if this wasn't the case we could use sklearn [LabelEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html).

In [15]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

In [16]:
model = Sequential()
model.add(Embedding(vocab_size, 32, input_length=maxlen))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

In [17]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 32)           320000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 373,301
Trainable params: 373,301
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=3, batch_size=64) 

C:\AIML\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 179s 7ms/step - loss: 0.4160 - acc: 0.8057 - val_loss: 0.2991 - val_acc: 0.8764
Epoch 2/3
25000/25000 [==============================] - 170s 7ms/step - loss: 0.2404 - acc: 0.9080 - val_loss: 0.3211 - val_acc: 0.8617
Epoch 3/3
25000/25000 [==============================] - 174s 7ms/step - loss: 0.1982 - acc: 0.9254 - val_loss: 0.3267 - val_acc: 0.8632


In [20]:
scores = model.evaluate(x_test, y_test, verbose=0) 
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 86.32%


## Retrive the output of each layer in keras for a given single test sample from the trained model you built

In [30]:
words["<P>"] = 0
id_to_word = {value:key for key,value in words.items()}
test_sample = ''
for id in x_test[0]:
    if id != 0:
        test_sample = test_sample + ' ' + id_to_word[id]
test_sample

" the wonder own as by is sequence i i and and to of hollywood br of down shouting getting boring of ever it sadly sadly sadly i i was then does don't close faint after one carry as by are be favourites all family turn in does as three part in another some to be probably with world and her an have faint beginning own as is sequence"

In [64]:
model.predict([[x_test[0]]])

array([[0.10407706]], dtype=float32)

In [ ]:
# From the prediction, as it's close to 0, it must be a negative review.

In [86]:
from keras import backend as K 
inp = model.input                                           
outputs = [layer.output for layer in model.layers]
functor = K.function([inp], outputs)

# Testing
#test = np.random.random(input_shape)[np.newaxis,...]
layer_outs = functor([[x_test[0]]])
#print layer_outs

In [87]:
layer_outs

[array([[[ 0.0478803 ,  0.03583616,  0.06444059, ...,  0.03969092,
           0.05401114,  0.01664785],
         [ 0.0478803 ,  0.03583616,  0.06444059, ...,  0.03969092,
           0.05401114,  0.01664785],
         [ 0.0478803 ,  0.03583616,  0.06444059, ...,  0.03969092,
           0.05401114,  0.01664785],
         ...,
         [ 0.03200355,  0.01085013, -0.01080003, ..., -0.00287445,
           0.10291281, -0.03244625],
         [ 0.03821947,  0.00990951, -0.01796627, ...,  0.03140771,
           0.0145089 ,  0.02776865],
         [ 0.02204306, -0.08721639, -0.01141045, ..., -0.01495066,
          -0.0104242 , -0.05562717]]], dtype=float32),
 array([[ 0.05192398, -0.00175801, -0.02073247, -0.09993102, -0.14294586,
         -0.02491867,  0.01711795, -0.01748232, -0.0177271 ,  0.0933497 ,
         -0.16036093, -0.03933778, -0.14137338, -0.01181587,  0.13795036,
          0.15730657, -0.02515659, -0.06703848,  0.02637838, -0.01771977,
         -0.04421455, -0.13356674,  0.00031794, 